In [21]:
import xml.etree.ElementTree as ET
import csv
import datetime

In [19]:
def get_text(token):
    
    texte = ""
    choice = ["corr", "expan", "reg"]
    w_token = ET.fromstring(token)
    
    if w_token.text:
        texte += w_token.text
    
    """for item in w_token.findall("./*"):
        # if item.tag == '{http://tei-c.org/ns/1.0}height' or item.tag == '{http://tei-c.org/ns/1.0}supplied':
        if item.tag == 'c' or item.tag == 'supplied':
            texte += str(item.text)
                        # S'il y a du texte après la balise fermante et avant
                        # le prochain enfant ou la balise fermante du <w>,
                        # on l'ajoute.
            if item.tail:
                texte += str(item.tail)
                        
                    # elif item.tag == '{http://tei-c.org/ns/1.0}lb':
        elif item.tag == 'lb':
            if item.tail:
                texte += str(item.tail)
                        
                    # Si l'enfant est un <choice>, on récupère le texte de son
                    # second enfant et on vérifie s'il y a du texte après le <choice>.
                    # elif item.tag == '{http://tei-c.org/ns/1.0}choice':
        elif item.tag == 'choice':
            for subitem in item:
                if subitem.tag in choice:
                    texte += str(subitem.text)
            if item.tail:
                texte += str(item.tail)
                            
        elif item.tag == 'add':
                        # On refait tous les tests.
            if item.find('.') == None :
                texte = str(item.text)
                            
            else:
                
                if item.text:
                    texte += str(item.text)
                            
                for subitem in item:
                    if subitem.tag == 'lb':
                        if subitem.tail:
                            texte += str(subitem.tail)
                    elif subitem.tag == 'choice':
                        texte += str(subitem[1].text)
                        if subitem.tail:
                            texte += str(subitem.tail)
                            """
    
    for item in w_token:
                
        # Si l'enfant est un <height>, on récupère son texte.
        if item.tag == 'height' or item.tag == 'supplied':
            texte += str(item.text)
            # S'il y a du texte après la balise fermante et avant
            # le prochain enfant ou la balise fermante du <w>,
            # on l'ajoute.
            if item.tail:
                texte += str(item.tail)
                        
        elif item.tag == 'lb':
            if item.tail:
                texte += str(item.tail)
                        
                    # Si l'enfant est un <choice>, on récupère le texte de son
                    # second enfant et on vérifie s'il y a du texte après le <choice>.
        elif item.tag == 'choice':
            texte += str(item[1].text)
            if item.tail:
                texte += str(item.tail)
                            
        elif item.tag == 'c':
            texte += item.text
            if item.tail:
                texte += str(item.tail)
                            
        elif item.tag == 'add':
            # On refait tous les tests.
            if item.find('.') == None :
                texte = str(item.text)
                            
            else:
                            
                if item.text:
                    texte += str(item.text)
                            
                for subitem in item:
                    if subitem.tag == 'lb':
                        if subitem.tail:
                            texte += str(subitem.tail)
                    elif subitem.tag == 'choice':
                        texte += str(subitem[1].text)
                        if subitem.tail:
                            texte += str(subitem.tail)
    
    return texte

In [26]:
def extraction(xml_entree, csv_simple, csv_concordancier, txt_stats):
    
    dico_tokens={}
    
    # colonnes des CSV:
    simple_cols = ["ID", "TOKEN", "LEMMES", "POS"]
    concord_cols = ["ID", "POS", "GAUCHE", "TOKEN", "DROIT"]
    
    # compteurs
    nb_total_tokens = 0
    pos_ambigus = 0
    pos_uniques = 0
    pos_inc = 0
    lemmes_ambigus = 0
    lemmes_uniques = 0
    lemmes_inc = 0
    
    # Pour que Python comprenne les éléments dont on parlera,
    # il faut lui donner la déclaration TEI, mais comme c'est
    # la seule qu'on utilisera, pas besoin de lui donner un préfixe.
    # ET.register_namespace('', "http://tei-c.org/ns/1.0")
    
    # On va chercher le fichier XML-TEI et on le lit.
    tree = ET.parse(xml_entree)
    root = tree.getroot()
    
    for word in root.findall('.//w'):
        dico_tokens[int(word.get('n'))] = get_text(ET.tostring(word))
        
    # On ouvre le CSV de sortie en mode "écriture", on y écrit le nom des colonnes.
    with open(csv_simple, 'w') as csv_file:
        csv_contenu = csv.DictWriter(csv_file, fieldnames = simple_cols, delimiter=";")
        csv_contenu.writeheader()
        
        # On boucle sur les éléments <w> du XML, dans l'ordre du fichier.
        # for word in root.findall('.//{http://tei-c.org/ns/1.0}w'):
        for word in root.findall('.//w'):
            
            nb_total_tokens += 1
        
            # On récupère les @n, @lemma et @pos dans les variables
            # "numero", "lemmes" et "pos"
            # et on crée la chaîne "texte", pour l'instant vide.
            numero = str(word.get('n'))
            lemmes = str(word.get('lemma'))
            pos = str(word.get('pos'))
            texte = get_text(ET.tostring(word))
            
            if '|' in lemmes:
                lemmes_ambigus += 1
                
                if '|' in pos:
                    pos_ambigus += 1
                elif pos=="Inconnu":
                    pos_inc += 1
                    
                csv_contenu.writerow(
                    {
                        "ID":numero,
                        "TOKEN":texte,
                        "LEMMES":lemmes,
                        "POS":pos
                    }
                )
                
            elif lemmes=="INC":
                lemmes_inc += 1
                
                if '|' in pos:
                    pos_ambigus += 1
                elif pos=="Inconnu":
                    pos_inc += 1
                    
                csv_contenu.writerow(
                    {
                        "ID":numero,
                        "TOKEN":texte,
                        "LEMMES":lemmes,
                        "POS":pos
                    }
                )
                
            else:
                lemmes_uniques += 1
                
                
    """with open(csv_concordancier, 'w') as csv_file:
        csv_contenu = csv.DictWriter(csv_file, fieldnames = concord_cols, delimiter=";")
        csv_contenu.writeheader()
        
        # On boucle sur les éléments <w> du XML, dans l'ordre du fichier.
        # for word in root.findall('.//{http://tei-c.org/ns/1.0}w'):
        for word in root.findall('.//w'):
        
            # On récupère les @n, @lemma et @pos dans les variables
            # "numero", "lemmes" et "pos"
            # et on crée la chaîne "texte", pour l'instant vide.
            numero = str(word.get('n'))
            lemmes = str(word.get('lemma'))
            pos = str(word.get('pos'))
            texte = get_text(ET.tostring(word))
            ["ID", "POSG", "GAUCHE", "TOKEN", "DROIT", "POSD"]
            if '|' in lemmes or '|' in pos or lemmes=="INC" or pos=="Inconnu":
                
                gauche = [dico_tokens[int(numero)-3], dico_tokens[int(numero)-2], dico_tokens[int(numero)-1]]
                droit = [dico_tokens[int(numero)+1], dico_tokens[int(numero)+2], dico_tokens[int(numero)+3]]
                
                csv_contenu.writerow(
                    {
                        "ID":numero,
                        "POS":pos,
                        "GAUCHE": " ".join(gauche),
                        "TOKEN": texte,
                        "DROITE":" ".join(droit)
                        
                    }
                )"""
        
    
    pourcentage_lemmes = lemmes_uniques * 100 / nb_total_tokens
    pourcentage_pos = pos_uniques * 100 / nb_total_tokens
    pourcentage_lemmes_inc = lemmes_inc * 100 / nb_total_tokens
    pourcentage_extraits = pos_ambigus * 100 / nb_total_tokens
        
    with open(txt_stats, "w") as file:
        file.write(str(datetime.datetime.now()))
        file.write(round(pourcentage_lemmes,2), "% de lemmes uniques.")
        file.write(round(pourcentage_pos,2), "% de POS uniques.")
        file.write(lemmes_inc, "lemmes inconnus, soit", round(pourcentage_lemmes_inc,2), "%.")

In [27]:
extraction('/home/erminea/Documents/CONDE/Rouille-TS/Rouille_19-lemmatise_div-ided.xml',
          '/home/erminea/Documents/CONDE/Rouille-TS/rouille_ambig_tableau.csv',
          '/home/erminea/Documents/CONDE/Rouille-TS/rouille_ambig_concord.csv',
          '/home/erminea/Documents/CONDE/Rouille-TS/stats.txt')

TypeError: write() takes exactly one argument (2 given)